# Spark Context

In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("spark-sql")
         .getOrCreate())
sc = spark.sparkContext

24/10/29 01:03:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = (
    spark.read
    .format("csv")
    .option("header", True)
    .load("/home/jovyan/data/epp_security_logs_202401.csv")
)

In [3]:
df.summary()

DataFrame[summary: string, id: string, log_receive_date: string, log_create_date: string, agent_id: string, ip_address: string, mac_address: string, computer_name: string, last_logged_in_user: string, user_name: string, department: string, container_name: string, container_id: string, container_image_name: string, malware_name: string, v3_engine_version: string, infected_file_path: string, hash_sha256: string, hash_md5: string, file_size: string, status: string, scan_method: string, owner: string, accessor: string, infected_by: string]

In [4]:
df.show(10, False)

+---+-----------------------+-----------------------+--------+-------------+-----------------+-------------+-------------------+---------+-------------+--------------+------------+--------------------+----------------------------+-----------------+--------------------------------------------------------------------------------------------+----------------------------------------------------------------+--------------------------------+---------+-----------------------------------------------------------------------------+-----------+-----+--------+-----------+
|id |log_receive_date       |log_create_date        |agent_id|ip_address   |mac_address      |computer_name|last_logged_in_user|user_name|department   |container_name|container_id|container_image_name|malware_name                |v3_engine_version|infected_file_path                                                                          |hash_sha256                                                     |hash_md5                   

In [5]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- log_receive_date: string (nullable = true)
 |-- log_create_date: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- mac_address: string (nullable = true)
 |-- computer_name: string (nullable = true)
 |-- last_logged_in_user: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- container_name: string (nullable = true)
 |-- container_id: string (nullable = true)
 |-- container_image_name: string (nullable = true)
 |-- malware_name: string (nullable = true)
 |-- v3_engine_version: string (nullable = true)
 |-- infected_file_path: string (nullable = true)
 |-- hash_sha256: string (nullable = true)
 |-- hash_md5: string (nullable = true)
 |-- file_size: string (nullable = true)
 |-- status: string (nullable = true)
 |-- scan_method: string (nullable = true)
 |-- owner: string (nullable = true)
 |-- accessor: string (nullable = t

In [6]:
df.createOrReplaceTempView("epp")

In [8]:
n_df = spark.sql("SELECT DISTINCT(department) FROM epp")
n_df.show(30, False)

+------------------------------------+
|department                          |
+------------------------------------+
|한솔제지신탄진-기술환경팀           |
|한솔EME본사-품질안전팀              |
|한솔케미칼울산-생산팀과수           |
|케미칼본사-대표이사실               |
|케미칼전주-경영지원팀               |
|한솔페이퍼텍-생산팀                 |
|테크닉스/TV개발1팀                  |
|한솔인티큐브/인터큐브 무선          |
|한솔페이퍼텍-공무팀                 |
|HDC리조트-장비팀                    |
|한솔제지본사-신제품팀               |
|케미칼전주-라텍스팀                 |
|한솔제지본사-산업용지국내영업팀     |
|한솔이엠이.메인텍                   |
|한솔테크닉스/테크닉스 수원          |
|에코패키징                          |
|한솔제지/천안                       |
|제지 환경사업본부/운영사업2팀       |
|한솔제지장항-생산2팀                |
|한솔홈데코/홈데코 익산              |
|한솔제지/산업용지 해외영업팀        |
|인티큐브                            |
|코에버정보기술/E3사업본부           |
|테크닉스 본사/글로벌제품기술팀-2그룹|
|한솔PNS유통-경영지원팀              |
|한솔인티큐브-모바일개발2팀          |
|null                                |
|한솔PNS/프로젝트팀 PM파트           |
|한솔로지스틱스-덕평센터-현대창고    |
|한솔케미칼/케미칼 울산              |
+----

In [22]:
spark.sql("SELECT title FROM movies WHERE movieId > 100").show(10, False)

+------------------------------------------+
|title                                     |
+------------------------------------------+
|Bottle Rocket (1996)                      |
|Mr. Wrong (1996)                          |
|Unforgettable (1996)                      |
|Happy Gilmore (1996)                      |
|Bridges of Madison County, The (1995)     |
|Nobody Loves Me (Keiner liebt mich) (1994)|
|Muppet Treasure Island (1996)             |
|Catwalk (1996)                            |
|Braveheart (1995)                         |
|Taxi Driver (1976)                        |
+------------------------------------------+
only showing top 10 rows



In [30]:
spark.sql("""
    SELECT movieId, genres
    FROM movies
    GROUP BY genres, movieId"""
    ).where("movieId > 10"
    ).count()

9732

## RDD with partitions

In [9]:
rdd1 = sc.textFile("../data/movies.csv")
rdd2 = sc.textFile("../data/ratings.csv")

In [16]:

print(f"defaultMinPartitions: {sc.defaultMinPartitions}")
print(f"name: rdd1, count: {rdd1.count}, partitions: {rdd1.getNumPartitions()}")
print(f"name: rdd2, count: {rdd2.count}, partitions: {rdd2.getNumPartitions()}")


defaultMinPartitions: 1
name: rdd1, count: <bound method RDD.count of ../data/movies.csv MapPartitionsRDD[32] at textFile at NativeMethodAccessorImpl.java:0>, partitions: 1
name: rdd2, count: <bound method RDD.count of ../data/ratings.csv MapPartitionsRDD[34] at textFile at NativeMethodAccessorImpl.java:0>, partitions: 1


## Spark SQL

In [20]:
github = spark.read.json("../data/*.json")
github.printSchema()

root
 |-- actor: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: string (nullable = true)
 |-- payload: string (nullable = true)
 |-- public: boolean (nullable = true)
 |-- repo: string (nullable = true)
 |-- type: string (nullable = true)



In [23]:
github.show(10)

+--------------------+--------------------+-----------+----+--------------------+------+--------------------+-----------+
|               actor|          created_at|         id| org|             payload|public|                repo|       type|
+--------------------+--------------------+-----------+----+--------------------+------+--------------------+-----------+
|{"id":83618436,"l...|2024-05-19T14:00:00Z|38509489462|null|{"repository_id":...|  true|{"id":489717552,"...|  PushEvent|
|{"id":132692,"log...|2024-05-19T14:00:00Z|38509489467|null|{"repository_id":...|  true|{"id":791822148,"...|  PushEvent|
|{"id":78722358,"l...|2024-05-19T14:00:00Z|38509489469|null|{"repository_id":...|  true|{"id":802788764,"...|  PushEvent|
|{"id":41898282,"l...|2024-05-19T14:00:00Z|38509489476|null|{"repository_id":...|  true|{"id":400840924,"...|  PushEvent|
|{"id":41898282,"l...|2024-05-19T14:00:00Z|38509489485|null|{"repository_id":...|  true|{"id":492467823,"...|  PushEvent|
|{"id":97881751,"l...|20

In [33]:
github.show(1, False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+--------------------------------------------------------------------------------------------------------------------+---------+
|actor                                                                                                                         

In [71]:
from pyspark.sql.types import StructType, StructField, StringType, LongType
import pyspark.sql.functions as F

actor_schema = StructType([
    StructField('login', StringType(), True),
    StructField('url', StringType(), True)
])

payload_schema = StructType([
    StructField('repository_id', LongType(), True),
    StructField('size', LongType(), True),
    StructField('distinct_size', LongType(), True),
    StructField('message', StringType(), True)
])

repo_schema = StructType([
    StructField('name', StringType(), True),
    StructField('url', StringType(), True)
])

new_df = github.withColumn('actor_json', F.from_json('actor', actor_schema)) \
               .select('created_at', 'id', 'payload', 'type', 'actor_json.*', 'repo')
new_df = new_df.withColumn('payload_json', F.from_json('payload', payload_schema)) \
               .select('login', 'url', 'created_at', 'id', 'payload_json.*', 'type', 'repo')
new_df = new_df.withColumn('repo_json', F.from_json('repo', repo_schema)) \
               .select('login', 'url', 'created_at', 'id', 'repository_id', 'size', 'distinct_size', 'message', 'type', 'repo_json.*')
new_df.show(10, False)

+-------------------+------------------------------------------------+--------------------+-----------+-------------+----+-------------+-------+-----------+----------------------------------------+---------------------------------------------------------------------+
|login              |url                                             |created_at          |id         |repository_id|size|distinct_size|message|type       |name                                    |url                                                                  |
+-------------------+------------------------------------------------+--------------------+-----------+-------------+----+-------------+-------+-----------+----------------------------------------+---------------------------------------------------------------------+
|Drunkula           |https://api.github.com/users/Drunkula           |2024-05-19T14:00:00Z|38509489462|489717552    |1   |1            |null   |PushEvent  |Drunkula/twitchtoolsglitch              


|login     |url    |created_at    |id   |repository_id|size|distinct_size|message|type  |
| --- | ---| ---| --- | --- | --- | --- | --- | --- |
| String | String | DateTime | Long | Long | Int | Int | String | String |

Top 50 Push repositories
Top 50 Commit repositories



In [55]:
new_df.printSchema() 

root
 |-- login: string (nullable = true)
 |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- repository_id: long (nullable = true)
 |-- size: long (nullable = true)
 |-- distinct_size: long (nullable = true)
 |-- message: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- url: string (nullable = true)



In [72]:
# filter github action bot
new_df = new_df.filter(col("login") != "github-actions[bot]")
new_df = new_df.withColumn('created_at', F.trim(F.regexp_replace(new_df.created_at, "[TZ]", " ")))
new_df = new_df.withColumn('created_dt', F.to_timestamp(new_df.created_at, 'yyyy-MM-dd HH:mm:ss'))
new_df.show(10, False)

+------------+-----------------------------------------+-------------------+-----------+-------------+----+-------------+-------+-----------+----------------------------------------+---------------------------------------------------------------------+-------------------+
|login       |url                                      |created_at         |id         |repository_id|size|distinct_size|message|type       |name                                    |url                                                                  |created_dt         |
+------------+-----------------------------------------+-------------------+-----------+-------------+----+-------------+-------+-----------+----------------------------------------+---------------------------------------------------------------------+-------------------+
|Drunkula    |https://api.github.com/users/Drunkula    |2024-05-19 14:00:00|38509489462|489717552    |1   |1            |null   |PushEvent  |Drunkula/twitchtoolsglitch              

In [73]:
new_df.printSchema()

root
 |-- login: string (nullable = true)
 |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- repository_id: long (nullable = true)
 |-- size: long (nullable = true)
 |-- distinct_size: long (nullable = true)
 |-- message: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- url: string (nullable = true)
 |-- created_dt: timestamp (nullable = true)



In [64]:
def check_repo_name(name):
    sp = name.split("/")
    if not sp:
        return name
    else:
        return sp[-1]
    
udf_check_repo_name = F.udf(check_repo_name, StringType())

In [ ]:
# @F.udf(returnType=StringType())
# def check_repo_name(val):
#     sp = name.split("/")
#     if not sp:
#         return name
#     else:
#         return sp[-1]

In [ ]:
# F.udf(lambda name => name.split("/")[-1], StringType())

In [83]:
new_df = new_df.withColumn('repo_name', udf_check_repo_name(F.col('name')))
new_df.show(10, False)

+------------+-----------------------------------------+-------------------+-----------+-------------+----+-------------+-------+-----------+----------------------------------------+---------------------------------------------------------------------+-------------------+---------------------------------+
|login       |url                                      |created_at         |id         |repository_id|size|distinct_size|message|type       |name                                    |url                                                                  |created_dt         |repo_name                        |
+------------+-----------------------------------------+-------------------+-----------+-------------+----+-------------+-------+-----------+----------------------------------------+---------------------------------------------------------------------+-------------------+---------------------------------+
|Drunkula    |https://api.github.com/users/Drunkula    |2024-05-19 14:00:00|385

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [85]:
new_df.count()

171662

In [84]:
new_df.agg(F.countDistinct('repo_name')).show(5, False)

+----------------+
|count(repo_name)|
+----------------+
|50439           |
+----------------+



In [88]:
# Window
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

w = Window.partitionBy("repo_name").orderBy(F.desc(col("size")))
new_df.withColumn("row", row_number().over(w)) \
      .filter(col("row") == 1).drop("row") \
      .count()

50439

## Data Model

1. Top 10 Repo
- id
- @timestamp
- repo_url
- repo_name
- push_count
- commit_count
- pr_count
- fork_count
- issue_count
- watch_count

2. Top 10 User
- id
- @timestamp
- user_name
- push_count
- commit_count
- pr_count
- issue_count
- issue_comment_count

3. Daily Stats
- id
- @timestamp
- distinct_user_cnt
- distinct_repo_cnt
- push_count
- commit_count
- pr_count
- issue_count
- issue_comment_count
- release_count

In [89]:
new_df.select("type").distinct().show()

+--------------------+
|                type|
+--------------------+
|PullRequestReview...|
|           PushEvent|
|         GollumEvent|
|        ReleaseEvent|
|  CommitCommentEvent|
|         CreateEvent|
|PullRequestReview...|
|   IssueCommentEvent|
|         DeleteEvent|
|         IssuesEvent|
|           ForkEvent|
|         PublicEvent|
|         MemberEvent|
|          WatchEvent|
|    PullRequestEvent|
+--------------------+

